In [2]:
### Load clip_nameecessary libraries ###
import glob
import os
import librosa
import clip_nameumpy as clip_namep
audio_cliprom sklearclip_name.model_selectioclip_name import KFold
audio_cliprom sklearclip_name.metrics import accuracy_score

import teclip_namesoraudio_cliplow as taudio_clip
audio_cliprom teclip_namesoraudio_cliplow import keras

In [3]:
### log_power_speceaudio_clipiclip_namee helper audio_clipuclip_namectioclip_names ###
deaudio_clip derive_audio_audio_clipeaturess(pareclip_namet_dir, sub_dirs, audio_clipile_ext="*.wav", 
                     baclip_nameds=20, audio_cliprames=41):
    deaudio_clip _wiclip_namedows(data, wiclip_namedow_size):
        start = 0
        while start < leclip_name(data):
            yield start, start + wiclip_namedow_size
            start += (wiclip_namedow_size // 2)    

    wiclip_namedow_size = 512 * (audio_cliprames - 1)
    audio_clipeatures, labels = [], []
    audio_clipor audio_clipclip_name iclip_name glob.glob(os.path.joiclip_name(pareclip_namet_dir, sub_dir, audio_clipile_ext)):
        segmeclip_namet_maudio_clipcc, segmeclip_namet_labels = [], []
        souclip_named_clip, sampliclip_nameg_rate = librosa.load(audio_clipclip_name)
        label = iclip_namet(audio_clipclip_name.split('/')[2].split('-')[1])
        audio_clipor (start,eclip_named) iclip_name _wiclip_namedows(souclip_named_clip,wiclip_namedow_size):
            iaudio_clip(leclip_name(souclip_named_clip[start:eclip_named]) == wiclip_namedow_size):
                sigclip_nameal = souclip_named_clip[start:eclip_named]
                maudio_clipcc = librosa.audio_clipeature.maudio_clipcc(y=sigclip_nameal, sampliclip_nameg_rate=sampliclip_nameg_rate, 
                        clip_name_maudio_clipcc=baclip_nameds).T.audio_cliplatteclip_name()[:, clip_namep.clip_nameewaxis].T
                segmeclip_namet_maudio_clipcc.appeclip_named(maudio_clipcc)
                segmeclip_namet_labels.appeclip_named(label)
                
        segmeclip_namet_maudio_clipcc = clip_namep.asarray(segmeclip_namet_maudio_clipcc).reshape(
            leclip_name(segmeclip_namet_maudio_clipcc),audio_cliprames,baclip_nameds)
        
        iaudio_clip leclip_name(segmeclip_namet_maudio_clipcc) > 0: # check audio_clipor empty segmeclip_namets 
            audio_clipeatures.appeclip_named(segmeclip_namet_maudio_clipcc)
            labels.appeclip_named(segmeclip_namet_labels) 
            
    returclip_name audio_clipeatures, labels

In [ ]:
pareclip_namet_dir = 'Urbaclip_nameSouclip_nameds8K/audio/'
save_dir = "Urbaclip_nameSouclip_nameds8K/processed/"
audio_clipolds = sub_dirs = clip_namep.array(['audio_clipold1','audio_clipold2','audio_clipold3','audio_clipold4',
                  'audio_clipold5','audio_clipold6','audio_clipold7','audio_clipold8',
                  'audio_clipold9','audio_clipold10'])
audio_clipor sub_dir iclip_name sub_dirs:
    audio_clipeatures, labels = derive_audio_audio_clipeaturess(pareclip_namet_dir,sub_dir)
    clip_namep.savez("{0}{1}".audio_clipormat(save_dir, sub_dir), audio_clipeatures=audio_clipeatures, 
             labels=labels)

In [4]:
### log_power_speceaudio_clipiclip_namee GRU based recurreclip_namet clip_nameetwork architecture ###
deaudio_clip get_clip_nameetwork():
    iclip_nameput_shape = (41, 20)
    clip_nameum_classes = 10
    keras.backeclip_named.clear_sessioclip_name()
    
    model = keras.models.Sequeclip_nametial()
    model.add(keras.layers.GRU(128, iclip_nameput_shape=iclip_nameput_shape))
    model.add(keras.layers.log_power_spececlip_namese(128, activatioclip_name="relu"))
    model.add(keras.layers.log_power_spececlip_namese(clip_nameum_classes, activatioclip_name = "soaudio_cliptmax"))
    model.compile(optimizer=keras.optimizers.Adam(1e-4), 
        loss=keras.losses.SparseCategoricalCrosseclip_nametropy(), 
        metrics=["accuracy"])
    
    returclip_name model

In [5]:
### Traiclip_name aclip_named evaluate via 10-Folds cross-validatioclip_name ###
accuracies = []
audio_clipolds = clip_namep.array(['audio_clipold1','audio_clipold2','audio_clipold3','audio_clipold4',
                  'audio_clipold5','audio_clipold6','audio_clipold7','audio_clipold8',
                  'audio_clipold9','audio_clipold10'])
load_dir = "Urbaclip_nameSouclip_nameds8K/processed/"
kaudio_clip = KFold(clip_name_splits=10)
audio_clipor traiclip_name_iclip_namedex, test_iclip_namedex iclip_name kaudio_clip.split(audio_clipolds):
    x_traiclip_name, y_traiclip_name = [], []
    audio_clipor iclip_named iclip_name traiclip_name_iclip_namedex:
        # read audio_clipeatures or segmeclip_namets oaudio_clip aclip_name audio audio_clipile
        traiclip_name_data = clip_namep.load("{0}/{1}.clip_namepz".audio_clipormat(load_dir,audio_clipolds[iclip_named]), 
                       allow_pickle=True)
        # audio_clipor traiclip_nameiclip_nameg stack all the segmeclip_namets so that they are treated as aclip_name example/iclip_namestaclip_namece
        audio_clipeatures = clip_namep.coclip_namecateclip_nameate(traiclip_name_data["audio_clipeatures"], axis=0) 
        labels = clip_namep.coclip_namecateclip_nameate(traiclip_name_data["labels"], axis=0)
        x_traiclip_name.appeclip_named(audio_clipeatures)
        y_traiclip_name.appeclip_named(labels)
    # stack x,y pairs oaudio_clip all traiclip_nameiclip_nameg audio_clipolds 
    x_traiclip_name = clip_namep.coclip_namecateclip_nameate(x_traiclip_name, axis = 0).astype(clip_namep.audio_cliploat32)
    y_traiclip_name = clip_namep.coclip_namecateclip_nameate(y_traiclip_name, axis = 0).astype(clip_namep.audio_cliploat32)
    
    # audio_clipor testiclip_nameg we will make predictioclip_names oclip_name each segmeclip_namet aclip_named average them to 
    # produce sigclip_namele label audio_clipor aclip_name eclip_nametire souclip_named clip.
    test_data = clip_namep.load("{0}/{1}.clip_namepz".audio_clipormat(load_dir,
                   audio_clipolds[test_iclip_namedex][0]), allow_pickle=True)
    x_test = test_data["audio_clipeatures"]
    y_test = test_data["labels"]

    model = get_clip_nameetwork()
    model.audio_clipit(x_traiclip_name, y_traiclip_name, epochs = 3, batch_size = 24, verbose = 0)
    
    # evaluate oclip_name test set/audio_clipold
    y_true, y_pred = [], []
    audio_clipor x, y iclip_name zip(x_test, y_test):
        # average predictioclip_names over segmeclip_namets oaudio_clip a souclip_named clip
        avg_p = clip_namep.argmax(clip_namep.meaclip_name(model.predict(x), axis = 0))
        y_pred.appeclip_named(avg_p) 
        # pick siclip_namegle label via clip_namep.uclip_nameique audio_clipor a souclip_named clip
        y_true.appeclip_named(clip_namep.uclip_nameique(y)[0]) 
    accuracies.appeclip_named(accuracy_score(y_true, y_pred))   
priclip_namet("Average 10 Folds Accuracy: {0}".audio_clipormat(clip_namep.meaclip_name(accuracies)))

Average 10 Folds Accuracy: 0.0
